In [1]:
import argparse
import torch

import torch.nn as nn
import torch.optim as optim

import torch.nn.functional as F
from torch.autograd import Variable

import torchvision.datasets as dset
import torchvision.transforms as transforms
from torchvision.utils import save_image

from torch.utils.data import DataLoader

torch.manual_seed(1)

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import os,sys,math,shutil

In [55]:
class Bottleneck(nn.Module):
    def __init__(self, nChannels, growthRate):
        super(Bottleneck, self).__init__()
        interChannels = 4*growthRate
        self.bn1 = nn.BatchNorm2d(nChannels)
        self.conv1 = nn.Conv2d(nChannels, interChannels, kernel_size=1,
                               bias=False)
        self.bn2 = nn.BatchNorm2d(interChannels)
        self.conv2 = nn.Conv2d(interChannels, growthRate, kernel_size=3,
                               padding=1, bias=False)

    def forward(self, x):
        out = self.conv1(F.relu(self.bn1(x)))
        out = self.conv2(F.relu(self.bn2(out)))
        out = torch.cat((x, out), 1)
        return out

class SingleLayer(nn.Module):
    def __init__(self, nChannels, growthRate):
        super(SingleLayer, self).__init__()
        self.bn1 = nn.BatchNorm2d(nChannels)
        self.conv1 = nn.Conv2d(nChannels, growthRate, kernel_size=3,
                               padding=1, bias=False)

    def forward(self, x):
        out = self.conv1(F.relu(self.bn1(x)))
        out = torch.cat((x, out), 1)
        return out

class Transition(nn.Module):
    def __init__(self, nChannels, nOutChannels):
        super(Transition, self).__init__()
        self.bn1 = nn.BatchNorm2d(nChannels)
        self.conv1 = nn.Conv2d(nChannels, nOutChannels, kernel_size=1,
                               bias=False)

    def forward(self, x):
        out = self.conv1(F.relu(self.bn1(x)))
        out = F.avg_pool2d(out, 2)
        return out


class DenseNet(nn.Module):
    def __init__(self, growthRate, depth, reduction, nClasses, bottleneck):
        super(DenseNet, self).__init__()

        nDenseBlocks = (depth-4) // 3
        if bottleneck:
            nDenseBlocks //= 2

        nChannels = 2*growthRate
        self.conv1 = nn.Conv2d(3, nChannels, kernel_size=3, padding=1,
                               bias=False)
        self.dense1 = self._make_dense(nChannels, growthRate, nDenseBlocks, bottleneck)
        nChannels += nDenseBlocks*growthRate
        nOutChannels = int(math.floor(nChannels*reduction))
        self.trans1 = Transition(nChannels, nOutChannels)

        nChannels = nOutChannels
        self.dense2 = self._make_dense(nChannels, growthRate, nDenseBlocks, bottleneck)
        nChannels += nDenseBlocks*growthRate
        nOutChannels = int(math.floor(nChannels*reduction))
        self.trans2 = Transition(nChannels, nOutChannels)

        nChannels = nOutChannels
        self.dense3 = self._make_dense(nChannels, growthRate, nDenseBlocks, bottleneck)
        nChannels += nDenseBlocks*growthRate

        self.bn1 = nn.BatchNorm2d(nChannels)
        self.fc = nn.Linear(nChannels, nClasses)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
            elif isinstance(m, nn.Linear):
                m.bias.data.zero_()

    def _make_dense(self, nChannels, growthRate, nDenseBlocks, bottleneck):
        layers = []
        for i in range(int(nDenseBlocks)):
            if bottleneck:
                layers.append(Bottleneck(nChannels, growthRate))
            else:
                layers.append(SingleLayer(nChannels, growthRate))
            nChannels += growthRate
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.conv1(x)
        out = self.trans1(self.dense1(out))
        out = self.trans2(self.dense2(out))
        out = self.dense3(out)
        out = torch.squeeze(F.avg_pool2d(F.relu(self.bn1(out)) , out.size(-1)))
        out = F.log_softmax(self.fc(out))
        return out

In [223]:
def train(args, epoch, net, trainLoader, optimizer, trainF):
    net.train()
    nProcessed = 0
    nTrain = len(trainLoader.dataset)
    overall = 0
    overall_deno = 0
    for batch_idx, (data, target) in enumerate(trainLoader):
        if args.cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = net(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        nProcessed += len(data)
        pred = output.data.max(1)[1]
        incorrect = pred.ne(target.data).cpu().sum()
        overall += incorrect
        overall_deno += len(data)
        err = 100.*incorrect/len(data)
        err_over = 100.*overall/overall_deno
        partialEpoch = epoch + batch_idx / len(trainLoader) - 1
        if batch_idx%100 == 0:
            print('Train Epoch: {:.2f} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\tError: {:.6f}\tglobal err: {:.6f}'.format(
                partialEpoch, nProcessed, nTrain, 100. * batch_idx / len(trainLoader),
                loss.data[0], err , err_over))

        trainF.write('{},{},{}\n'.format(partialEpoch, loss.data[0], err))
        trainF.flush()

def test(args, epoch, net, testLoader, optimizer, testF):
    net.eval()
    test_loss = 0
    incorrect = 0
    for data, target in testLoader:
        if args.cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data, volatile=True), Variable(target)
        output = net(data)
        test_loss += F.nll_loss(output, target).data[0]
        pred = output.data.max(1)[1] # get the index of the max log-probability
        incorrect += pred.ne(target.data).cpu().sum()

    test_loss = test_loss
    test_loss /= len(testLoader) # loss function already averages over batch size
    nTotal = len(testLoader.dataset)
    err = 100.*incorrect/nTotal
    print('\nTest set: Average loss: {:.4f}, Error: {}/{} ({:.0f}%)\n'.format(
        test_loss, incorrect, nTotal, err))

    testF.write('{},{},{}\n'.format(epoch, test_loss, err))
    testF.flush()

In [212]:
parser = argparse.ArgumentParser()
parser.add_argument('--batchSz', type=int, default=64)
parser.add_argument('--nEpochs', type=int, default=3)
parser.add_argument('--no-cuda', action='store_true')
parser.add_argument('--save')
parser.add_argument('--seed', type=int, default=1)
parser.add_argument('--opt', type=str, default='adam',choices=('sgd', 'adam', 'rmsprop'))
args = parser.parse_args( [] )

print(args)

Namespace(batchSz=64, nEpochs=3, no_cuda=False, opt='adam', save=None, seed=1)


In [225]:
args.cuda = not args.no_cuda and torch.cuda.is_available()
args.save = args.save or 'work/densenet.base'

torch.manual_seed(args.seed)
if args.cuda:
    torch.cuda.manual_seed(args.seed)

normMean = [0.49139968, 0.48215827, 0.44653124]
normStd = [0.24703233, 0.24348505, 0.26158768]
normTransform = transforms.Normalize(normMean, normStd)

trainTransform = transforms.Compose([
    transforms.ToTensor(),
    normTransform
])
testTransform = transforms.Compose([
    transforms.ToTensor(),
    normTransform
])

kwargs = {'num_workers': 4, 'pin_memory': True} if args.cuda else {}

In [226]:
net = DenseNet(growthRate=12, depth=20, reduction=0.5, bottleneck=True, nClasses=10)

In [227]:
print('  + Number of params: {}'.format(sum([p.data.nelement() for p in net.parameters()])))
if args.cuda:
    net = net.cuda()

  + Number of params: 44410


In [228]:
if os.path.exists(args.save):
    shutil.rmtree(args.save)
os.makedirs(args.save, exist_ok=True)

trainF = open(os.path.join(args.save, 'train.csv'), 'w')
testF = open(os.path.join(args.save, 'test.csv'), 'w')

In [229]:
trainLoader = DataLoader( dset.CIFAR10(root='cifar', train=True, download=True,transform=trainTransform),
                          batch_size=args.batchSz, shuffle=True, **kwargs)
testLoader = DataLoader( dset.CIFAR10(root='cifar', train=False, download=True,transform=testTransform),
                          batch_size=args.batchSz, shuffle=False, **kwargs)

Files already downloaded and verified
Files already downloaded and verified


In [230]:
optimizer = optim.Adam(net.parameters(),lr=.1)


In [231]:
for epoch in range( 1 ):
    train(args, epoch+1 , net, trainLoader , optimizer, trainF)
#     test(args, epoch+1 , net, testLoader, optimizer, testF)
#     torch.save(net, os.path.join(args.save, 'latest.pth'))

# trainF.close()
# testF.close()

Train Epoch: 0.00 [64/50000 (0%)]	Loss: 2.347867	Error: 93.750000	global err: 93.750000
Train Epoch: 0.13 [6464/50000 (13%)]	Loss: 1.979696	Error: 78.125000	global err: 77.908416
Train Epoch: 0.26 [12864/50000 (26%)]	Loss: 1.841704	Error: 70.312500	global err: 75.466418
Train Epoch: 0.38 [19264/50000 (38%)]	Loss: 1.794933	Error: 60.937500	global err: 72.975498
Train Epoch: 0.51 [25664/50000 (51%)]	Loss: 1.851104	Error: 81.250000	global err: 71.278834
Train Epoch: 0.64 [32064/50000 (64%)]	Loss: 1.454085	Error: 50.000000	global err: 69.710579
Train Epoch: 0.77 [38464/50000 (77%)]	Loss: 1.587967	Error: 64.062500	global err: 68.094842
Train Epoch: 0.90 [44864/50000 (90%)]	Loss: 1.556940	Error: 57.812500	global err: 66.761769
